### **6.11 - [Practica] Sharing (big?) data with XComs**

Ok, en este video vamos a jugar con XCOMs y descubrir las formas de empujar y tirar de ellos.  Desde tu editor de código, comprueba que estás en la carpeta airflow-materials/airflow-section-6 y abre el DAG "xcom_dag.py". 

<center><img src="https://i.postimg.cc/130zfZp9/a1049.png"></center>
<center><img src="https://i.postimg.cc/gJ7zfcPK/a1050.png"></center>
<center><img src="https://i.postimg.cc/8kFMsjsN/a1051.png"></center>

En este DAG he implementado diferentes formas de usar XCOMs. Tenemos las tareas t0, t1, t2 hasta t6, así como una tarea especial que es un BranchPythonOperator para elegir entre las tareas t3 o t4 dependiendo del valor de un XCOM. 

**`Empecemos por t0`**. Si echas un vistazo a la función "push_xcom_with_return", puedes ver que creamos un xcom devolviendo el valor 'my_returned_xcom'. Significa que la key de este XCOM será igual al string "return_value". ¿Sabéis qué? Vamos a disparar el DAG ahora mismo para que veas claramente de lo que estoy hablando. Abre tu terminal y comprueba que estás en la carpeta airflow-materials/airflow-section-6 y escribe el comando "docker-compose -f docker-compose-CeleryExecutor.yml up -d". Enter. Ahora, Airflow se está ejecutando, en su navegador web escriba localhost:8080.  Desde ahí, activa el toggle del DAG "xcom_dag" y empieza a refrescar la página hasta que termine el DAGRun.  Bien, ahora si hacemos click en "admin" y "xcoms", obtendremos la lista de todos los XCOMs almacenados en la metadata  database. 

<center><img src="https://i.postimg.cc/k5fXvF1z/a1052.png"></center>

Aquí tenemos dos xcoms, pero vamos a centrarnos en el primero que corresponde a la tarea t0. Así que tenemos la key igual a 'return_value' ya que el XCOM ha sido empujado devolviendo un valor. El valor es 'my_returned_xcom'. El timestamp corresponde a la fecha en la que se ha enviado el XCOM. La execution date corresponde a la execution date de su DAG. El task id y el dag id te dan información sobre la tarea y el DAG a partir del cual se ha creado el XCOM.  

Muy bien, una cosa que debe tener en cuenta es que Airflow no limpiará automáticamente sus XCOM. Depende de ti hacer el trabajo y crear un proceso automático para limpiarlo. De lo contrario, en algún momento desperdiciarás recursos de espacio de tu metadata database. 

Ok, volvamos a nuestro DAG. **`Una vez que el xcom ha sido empujado por la tarea t0, la tarea t1 intentará tirar de él`**. Si echamos un vistazo a la función "get_pushed_xcom_with_return", podemos ver que necesitamos el "contexto de la instancia de la tarea" que se proporciona en los parámetros.  Este parámetro se hace disponible poniendo a "true" el parámetro "provide_context" del PythonOperator. A continuación, accedemos al objeto instancia de tarea en el contexto y llamamos al método xcom_pull. Como no tenemos ninguna key, especificamos la task id de la que queremos el XCOM que es "t0". Luego se imprime el valor. Ahora, desde la interfaz de usuario, si hacemos clic en "DAGs", "xcom _dags", luego seleccionamos la tarea t1 y "View log".  El valor"'my_returned_xcom" del XCOM ha sido bien extraído (pulled) como se muestra en el output aquí. 

<center><img src="https://i.postimg.cc/FzJrmtrX/a1053.png"></center>

Puedes pensar "un momento, tenemos una key que es "return_value", ¿por qué no usarla?". Eso es cierto y, de hecho, la key es utilizada implícitamente por Airflow. Cuando no se especifica ninguna key en el método xcom_pull, el valor por defecto para esa key es "return_value".  Así que especificar "return_value" no es necesario.  

Bien, hemos sido capaces de empujar un XCOM devolviendo su valor. Hemos extraído el XCOM a través del parámetro 'task id', veamos otras dos formas de empujar y extraer XCOMs. **`En nuestro DAG, la tarea t2 empujará un XCOM con la task id correspondiente a la siguiente tarea a ejecutar`**. Si consultamos la función "push_next_task", seguimos necesitando acceder al "contexto de la instancia de la tarea", pero esta vez, en lugar de devolver un valor (returning a value) para crear un XCOM, llamamos al método xcom_push con la key 'next_task' y el valor 't3'. Luego, en el BranchPythonOperator, se llama a la función "get_next_task" donde se extrae el XCOM especificando su key 'next _task'.  Finalmente se retorna el valor de ese XCOM para que el BranchPythonOperator elija la tarea correcta a ejecutar.  Así que nada más difícil, sólo hemos añadido una key al XCOM que empujamos y tiramos. Volviendo a la UI, en la vista de XCOMs, tenemos el XCOM creado a partir de la tarea t2 con la key 'next_task' y el valor 't3' como era de esperar.  

<center><img src="https://i.postimg.cc/hGStm3rV/a1054.png"></center>

Ahora bien, si vamos a la tree view de ‘xcom_dag’, y haga clic en la tarea "branching", a continuación, "View log", podemos ver que siguió la branch a partir de la tarea t3 como se define desde el XCOM tirado.  

<center><img src="https://i.postimg.cc/qgVkvmqh/a1055.png"></center>

Muy bien. **`Ya les dije en el vídeo anterior que podemos tirar de múltiples XCOMs a la vez especificando diferentes task ids en el método xcom_pull. Si vas al DAG, eso es lo que hacemos en la tarea t5`**. En la función de python "get_multiple_xcoms", extraemos los valores más recientes de los XCOM creados por las tareas t0 y t2. De nuevo, en la tree view del DAG, si hacemos clic en la tarea 5, luego en "View log". De la salida obtenemos una tupla con dos valores, "my_returned_XCOM" y "t3", tal y como se definen en los XCOM. 

<center><img src="https://i.postimg.cc/XNTF54yF/a1056.png"></center>

Podemos comprobarlo mirando la vista de los XCOMs, y los valores son los esperados.  Ok, volviendo a nuestro DAG tengo dos preguntas para ti antes de seguir adelante.  **`La primera pregunta es, ¿por qué he establecido la trigger rule de la tarea t5 en "one_success"?`**  Si no sabes la respuesta, deberías echar un vistazo al vídeo sobre las trigger rules. **`La segunda pregunta es ¿cuál será la salida de la tarea t6?`** Pausa el vídeo ahora mismo, intenta responder a estas dos preguntas y vuelve cuando estés listo.  

Ok, la respuesta a la primera pregunta es porque una de las tareas anteriores de t5 puede ser omitida por el BranchPythonOperator, si dejo la trigger rule por defecto que es all_success, la tarea t5 será omitida también.  ¿Por qué?  Porque cuando se omite una tarea upstream, por defecto, el estado se propaga a las tareas downstream a menos que la regla trigger sea diferente de all_sucess o all_failed. De nuevo, si no has entendido bien, te recomiendo encarecidamente que eches un vistazo al vídeo sobre las trigger rules.  

<center><img src="https://i.postimg.cc/sXxf6QbX/a1057.png"></center>

La respuesta a la segunda pregunta se puede encontrar desde la interfaz de usuario.  Aquí, usamos la variable predefinida ti para extraer el xcom con la key 'next_task' del parámetro templated bash_command. En la interfaz de usuario, si volvemos a la tree view del DAG y hacemos clic en la tarea t6 y luego en "Rendered", obtenemos el output "value from xcom t3".  

<center><img src="https://i.postimg.cc/ZKpNF2rQ/a1058.png"></center>

Así que como ves, también puedes utilizar xcoms dentro de tus templates.  Ya te dije que Airflow es muy potente.  Muy bien, lo último que quiero mostrarte es lo que ocurre si intentamos compartir un dataframe grande entre dos tareas usando XCOMs. Primero en tu editor de código, abre el archivo xcom_big_dag.py. En este DAG, sólo tenemos dos tareas donde la primera crea un gran dataframe panda con un tamaño aproximado de 200 megabytes.  Si eres un panda adicto, puedes comprobar lo que hace la función, pero básicamente solo creo cadenas aleatorias en un dataframe que devuelvo para crear el XCOM.  Así que vamos a ver cuál es el resultado de hacer eso. Desde la UI, haz click en Dags y activa el toggle del DAG xcom_big_dag.  Ahora empieza a refrescar hasta que el DAGRun haya terminado. Y funciona. De nuevo como mejor práctica nunca deberías hacer eso.  En realidad, ni siquiera deberías procesar datos con panda dentro de Airflow.  Ese es el trabajo de un framework de procesamiento de datos como Spark.  No olvides que Airflow es un orquestador.  Muy bien, abre tu terminal y detén el contenedor docker escribiendo "docker-compose - f docker-compose-CeleryExecutor.yml down".  Ahora vamos a tomar un breve descanso y nos vemos para el próximo video.  